In [ ]:
import sys
sys.path.insert(0, '/root/thomas/github/Mask_RCNN/')

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
import urllib2
import shutil
COCO_MODEL_URL = "https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5"


In [ ]:
def download_trained_weights(coco_model_path, verbose=1):
    """Download COCO trained weights from Releases.

    coco_model_path: local path of COCO trained weights
    """
    if verbose > 0:
        print("Downloading pretrained model to " + coco_model_path + " ...")
    resp = urllib2.urlopen(COCO_MODEL_URL)
    with open(coco_model_path, 'wb') as out:
        shutil.copyfileobj(resp, out)
    if verbose > 0:
        print("... done downloading pretrained model!")

In [ ]:
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

from config import Config
import utils
import model as modellib
import visualize
from model import log

from pycocotools.coco import COCO
from pycocotools import mask as maskUtils


%matplotlib inline 

# Root directory of the project

ROOT_DIR = '/root/data/models/mask_rcnn/run0/'
if not os.path.isdir(ROOT_DIR):
    os.makedirs(ROOT_DIR)
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
print(COCO_MODEL_PATH)
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    download_trained_weights(COCO_MODEL_PATH)

In [ ]:
class ShapesConfig(Config):
    """Configuration for traininspect_model.ipynb This notebook goes in depth into the steps performed to detect and segment objects. It provides visualizations of every step of the pipeline.ing on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "fish"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 3 shapes

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 512)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 64

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 1000

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
config = ShapesConfig()
config.display()

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [ ]:
# Which weights to start with?
init_with = "imagenet"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last()[1], by_name=True)

# write own class

In [ ]:
import json
from PIL import Image, ImageDraw

In [ ]:
# class FishDataset(utils.Dataset):
#     """Generates the shapes synthetic dataset. The dataset consists of simple
#     shapes (triangles, squares, circles) placed randomly on a blank surface.
#     The images are generated on the fly. No file access required.
#     """

#     def load_fish(self, annotations_path, height, width):
#         """Generate the requested number of synthetic images.
#         count: number of images to generate.
#         height, width: the size of the generated images.
#         """
#         # Add classes
#         self.add_class("fish", 1, "fish")
        
#         all_images = json.load(open(annotations_path))

#         # Add images
#         # Generate random specifications of images (i.e. color and
#         # list of shapes sizes and locations). This is more compact than
#         # actual images. Images are generated on the fly in load_image().
#         for (i, img) in enumerate(all_images):
#             # bg_color, shapes = self.random_image(height, width)
#             self.add_image("fish", 
#                            image_id=i, 
#                            path=img['path'],
#                            width=width, 
#                            height=height,
#                            bbox=img['bbox'],
#                            annotations=img['segmentation'])

#     def load_image(self, image_id):
#         """Generate an image from the specs of the given image ID.
#         Typically this function loads the image from a file, but
#         in this case it generates the image on the fly from the
#         specs in image_info.
#         """
#         info = self.image_info[image_id]
#         image = np.array(Image.open(info['path']))
#         return image

#     def image_reference(self, image_id):
#         """Return the shapes data of the image."""
#         info = self.image_info[image_id]
#         if info["source"] == "fish":
#             return info["fish"]
#         else:
#             super(self.__class__).image_reference(self, image_id)

#     def load_mask(self, image_id):
#         """Generate instance masks for shapes of the given image ID.
#         """
#         info = self.image_info[image_id]        
#         polygons = info['annotations']
#         count = len(polygons)
#         masks = np.zeros([info['height'], info['width'], count], dtype=np.uint8)
#         for (i, polygon) in enumerate(info['annotations']):
#             polygon = [(i1, info['height'] - i2) for (i1, i2) in polygon]
#             tupled = [tuple(p) for p in polygon]
#             img = Image.new('L', (info['width'], info['height']), 0)
#             ImageDraw.Draw(img).polygon(tupled, outline=1, fill=1)
#             mask = np.array(img)
#             masks[:, :, i] = mask
#         # Map class names to class IDs.
#         class_ids = np.array([1]*count)
#         return masks, class_ids.astype(np.int32)


In [ ]:
class FishDataset(utils.Dataset):
    """Generates the shapes synthetic dataset. The dataset consists of simple
    shapes (triangles, squares, circles) placed randomly on a blank surface.
    The images are generated on the fly. No file access required.
    """

    def load_fish(self, annotations_path):
        """Generate the requested number of synthetic images.
        count: number of images to generate.
        height, width: the size of the generated images.
        """
        # Add classes
        self.add_class("fish", 1, "fish")
        
        all_images = json.load(open(annotations_path))
        
        # Add images
        # Generate random specifications of images (i.e. color and
        # list of shapes sizes and locations). This is more compact than
        # actual images. Images are generated on the fly in load_image().
        for (i, img) in enumerate(all_images):
            # bg_color, shapes = self.random_image(height, width)
            if not isinstance(img, list):
                self.add_image("fish", 
                               image_id=i, 
                               path=img['path'],
                               bbox=img['bboxes'],
                               masks=img['masks'])

    def load_image(self, image_id):
        """Generate an image from the specs of the given image ID.
        Typically this function loads the image from a file, but
        in this case it generates the image on the fly from the
        specs in image_info.
        """
        info = self.image_info[image_id]
        image = np.array(Image.open(info['path']))
        return image

    def image_reference(self, image_id):
        """Return the shapes data of the image."""
        info = self.image_info[image_id]
        if info["source"] == "fish":
            return info["fish"]
        else:
            super(self.__class__).image_reference(self, image_id)

    def load_mask(self, image_id):
        """Generate instance masks for shapes of the given image ID.
        """
#         info = self.image_info[image_id]        
#         polygons = info['annotations']
#         count = len(polygons)
#         masks = np.zeros([info['height'], info['width'], count], dtype=np.uint8)
#         for (i, polygon) in enumerate(info['annotations']):
#             polygon = [(i1, info['height'] - i2) for (i1, i2) in polygon]
#             tupled = [tuple(p) for p in polygon]
#             img = Image.new('L', (info['width'], info['height']), 0)
#             ImageDraw.Draw(img).polygon(tupled, outline=1, fill=1)
#             mask = np.array(img)
#             masks[:, :, i] = mask
        info = self.image_info[image_id]
        masks = maskUtils.decode(info['masks'])
        # Map class names to class IDs.
        class_ids = np.array([1]*masks.shape[2])
        return masks, class_ids.astype(np.int32)


In [ ]:
all_images = json.load(open('/root/data/blender_data/test/training/validation_low_rez/labels.json'))

In [ ]:
all_images[0]

In [ ]:
dataset_train = FishDataset()
dataset_train.load_fish('/root/data/blender_data/test/training/train_low_rez/labels.json')
dataset_train.prepare()

In [ ]:
dataset_val = FishDataset()
dataset_val.load_fish('/root/data/blender_data/test/training/validation_low_rez/labels.json')
dataset_val.prepare()

In [ ]:
# *** This training schedule is an example. Update to your needs ***

# Training - SImageDrawtage 1
print("Training network heads")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=40,
            layers='all')